# Pedestiran Counts Virtual Sensor

In [1]:
import pandas as pd
import geopy
from geopy import distance

In [2]:
df = pd.read_csv('pedestrian_feb.csv')
df2 = pd.read_csv('Pedestrian_Counting_System_-_Sensor_Locations.csv')

In [16]:
#df3_new[df3_new['date_time'] == '17/02/2019 19:00']

## Distance Functions

In [3]:
def within(df, target, radius, g):
    
    temp = []
    
    for x in range(len(target)):
        # Use try-except for error handling
        try:
            # Find coordinate1,2 calculate distance in km
            coords_1 = (df["Latitude"][g], df["Longitude"][g])
            coords_2 = (target["latitude"][x], target["longitude"][x])
            dist = geopy.distance.distance(coords_1, coords_2).km

            if dist <= radius:
                temp.append(target.iloc[x])
        except:
            pass
    return temp

In [4]:
def within_d(df, target, radius, g):
    
    distances = []
    
    for x in range(len(target)):
        # Use try-except for error handling
        try:
            # Find coordinate1,2 calculate distance in km
            coords_1 = (df["Latitude"][g], df["Longitude"][g])
            coords_2 = (target["latitude"][x], target["longitude"][x])
            dist = geopy.distance.distance(coords_1, coords_2).km

            if dist <= radius:
                distances.append(dist)
        except:
            pass
    return distances

## Assign Lat, Lon for each sensor

In [19]:
df.head(2)

,date_time,day,hourly_counts,id,mdate,month,sensor_id,sensor_name,time,year
0,17/02/2019 19:00,Sunday,1513,2566097,17,February,4,Town Hall (West),19,2019
1,17/02/2019 19:00,Sunday,200,2566098,17,February,17,Collins Place (South),19,2019


In [20]:
df2 = df2.iloc[:,[0,8,9]]

In [21]:
df2.head(2)

,sensor_id,latitude,longitude
0,1,-37.813494,144.965153
1,2,-37.813807,144.965167


In [22]:
# Merge on sensor id to get lat, long for each location
df3 = pd.merge(df, df2, on='sensor_id', how='outer').iloc[:50000,[0,2,6,7,10,11]]

In [23]:
df3.head()

,date_time,hourly_counts,sensor_id,sensor_name,latitude,longitude
0,17/02/2019 19:00,1513.0,4,Town Hall (West),-37.81488,144.966088
1,17/02/2019 20:00,1234.0,4,Town Hall (West),-37.81488,144.966088
2,17/02/2019 21:00,1026.0,4,Town Hall (West),-37.81488,144.966088
3,17/02/2019 22:00,693.0,4,Town Hall (West),-37.81488,144.966088
4,17/02/2019 23:00,388.0,4,Town Hall (West),-37.81488,144.966088


## Create Virtual Sensor

### <font color='blue'>Sensor A - Collins St</font>


In [24]:
sensor1 = pd.DataFrame([{'Latitude':-37.817771,'Longitude':144.957951}])
sensor1

,Latitude,Longitude
0,-37.817771,144.957951


### Nearest sensors

In [25]:
# Find Sensor information and Distance
df4 = pd.concat([pd.DataFrame(within(sensor1, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor1, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [26]:
# Nearest 3 sensors
df4 = df4.sort_values('distance').head(3)
df4 = df4.reset_index(drop=True)
df4.head(3)

,sensor_id,latitude,longitude,distance
0,36.0,-37.816525,144.961211,0.318577
1,24.0,-37.818880,144.954492,0.328479
2,23.0,-37.819093,144.954527,0.335229


In [27]:
df_nearest = df3[(df3['sensor_id'] == df4['sensor_id'][0]) |
                 (df3['sensor_id'] == df4['sensor_id'][1]) |
                 ((df3['sensor_id'] == df4['sensor_id'][2]))]

In [28]:
d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['hourly_counts']*(df4['distance'][0]/df4['distance'].sum())
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][1]]['hourly_counts']*(df4['distance'][1]/df4['distance'].sum())
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][2]]['hourly_counts']*(df4['distance'][2]/df4['distance'].sum())

In [29]:
sensor_a = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

In [30]:
sensor_a.head(1)

,date_time,hourly_counts
0,17/02/2019 19:00,457.0


In [31]:
sensor_a['latitude']= sensor1['Latitude'][0]
sensor_a['longitude']= sensor1['Longitude'][0]
sensor_a['sensor_name']= 'Collin St - Virtual'
sensor_a['sensor_id']= 101
sensor_a.head()

,date_time,hourly_counts,latitude,longitude,sensor_name,sensor_id
0,17/02/2019 19:00,457.0,-37.817771,144.957951,Collin St - Virtual,101
1,17/02/2019 20:00,351.0,-37.817771,144.957951,Collin St - Virtual,101
2,17/02/2019 21:00,314.0,-37.817771,144.957951,Collin St - Virtual,101
3,17/02/2019 22:00,198.0,-37.817771,144.957951,Collin St - Virtual,101
4,17/02/2019 23:00,131.0,-37.817771,144.957951,Collin St - Virtual,101


### Append Existing DF

In [32]:
df3_new = df3.append(sensor_a, sort=False).reset_index(drop=True)
df2 = df2.append(sensor_a.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

### <font color='blue'>Sensor B - Bourke St</font>

In [33]:
sensor2 = pd.DataFrame([{'Latitude':-37.81447,'Longitude':144.96197}])
sensor2

,Latitude,Longitude
0,-37.81447,144.96197


### Nearest sensors

In [34]:
# Find Sensor information and Distance
df4 = pd.concat([pd.DataFrame(within(sensor2, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor2, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [35]:
# Nearest 3 sensors
df4 = df4.sort_values('distance').head(3)
df4 = df4.reset_index(drop=True)
df4.head(3)

,sensor_id,latitude,longitude,distance
0,52.0,-37.812522,144.961940,0.216278
1,36.0,-37.816525,144.961211,0.237719
2,56.0,-37.812348,144.961533,0.238676


In [36]:
df_nearest = df3_new[(df3_new['sensor_id'] == df4['sensor_id'][0]) |
                 (df3_new['sensor_id'] == df4['sensor_id'][1]) |
                 ((df3_new['sensor_id'] == df4['sensor_id'][2]))]

In [37]:
d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['hourly_counts']*(df4['distance'][0]/df4['distance'].sum())
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][1]]['hourly_counts']*(df4['distance'][1]/df4['distance'].sum())
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][2]]['hourly_counts']*(df4['distance'][2]/df4['distance'].sum())

In [38]:
sensor_b = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

In [39]:
sensor_b.head(1)

,date_time,hourly_counts
0,17/02/2019 19:00,344.0


In [40]:
sensor_b['latitude']= sensor2['Latitude'][0]
sensor_b['longitude']= sensor2['Longitude'][0]
sensor_b['sensor_name']= 'Bourke St - Virtual'
sensor_b['sensor_id']= 102
sensor_b.head()

,date_time,hourly_counts,latitude,longitude,sensor_name,sensor_id
0,17/02/2019 19:00,344.0,-37.81447,144.96197,Bourke St - Virtual,102
1,17/02/2019 20:00,283.0,-37.81447,144.96197,Bourke St - Virtual,102
2,17/02/2019 21:00,230.0,-37.81447,144.96197,Bourke St - Virtual,102
3,17/02/2019 22:00,149.0,-37.81447,144.96197,Bourke St - Virtual,102
4,17/02/2019 23:00,105.0,-37.81447,144.96197,Bourke St - Virtual,102


### Append Existing DF

In [41]:
df3_new = df3_new.append(sensor_b, sort=False).reset_index(drop=True)

In [42]:
df2 = df2.append(sensor_b.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

### <font color='blue'>Sensor C - sensor 13</font>

In [43]:
sensor3 = pd.DataFrame([{'Latitude':-37.812397,'Longitude':144.956526}])
sensor3

,Latitude,Longitude
0,-37.812397,144.956526


### Nearest sensors

In [44]:
# Find Sensor information and Distance
df4 = pd.concat([pd.DataFrame(within(sensor3, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor3, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [46]:
# Nearest 3 sensors
df4 = df4.sort_values('distance').iloc[0:4,:]
df4 = df4.reset_index(drop=True)
df4.head(3)

,sensor_id,latitude,longitude,distance
0,56.0,-37.812348,144.961533,0.440938
1,52.0,-37.812522,144.961940,0.476943
2,51.0,-37.808418,144.959063,0.494924


In [47]:
df_nearest = df3_new[(df3_new['sensor_id'] == df4['sensor_id'][0]) |
                 (df3_new['sensor_id'] == df4['sensor_id'][1]) |
                 ((df3_new['sensor_id'] == df4['sensor_id'][2]))]

In [48]:
d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['hourly_counts']*(df4['distance'][0]/df4['distance'].sum())
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][1]]['hourly_counts']*(df4['distance'][1]/df4['distance'].sum())
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][2]]['hourly_counts']*(df4['distance'][2]/df4['distance'].sum())

In [49]:
sensor_c = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

In [50]:
sensor_c.head(1)

,date_time,hourly_counts
0,17/02/2019 19:00,249.0


In [51]:
df2 = df2.drop(df.index[[12]]).reset_index(drop=True)

In [52]:
sensor_c['latitude']= sensor3['Latitude'][0]
sensor_c['longitude']= sensor3['Longitude'][0]
sensor_c['sensor_name']= 'Sensor13 - Virtual'
sensor_c['sensor_id']= 13
sensor_c.head()

,date_time,hourly_counts,latitude,longitude,sensor_name,sensor_id
0,17/02/2019 19:00,249.0,-37.812397,144.956526,Sensor13 - Virtual,13
1,17/02/2019 20:00,211.0,-37.812397,144.956526,Sensor13 - Virtual,13
2,17/02/2019 21:00,172.0,-37.812397,144.956526,Sensor13 - Virtual,13
3,17/02/2019 22:00,112.0,-37.812397,144.956526,Sensor13 - Virtual,13
4,17/02/2019 23:00,85.0,-37.812397,144.956526,Sensor13 - Virtual,13


### Append Existing DF

In [53]:
df3_new = df3_new.append(sensor_c, sort=False).reset_index(drop=True)

In [54]:
df2 = df2.append(sensor_c.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

### <font color='blue'>Sensor D - Queen St</font>

In [55]:
sensor4 = pd.DataFrame([{'Latitude':-37.810979,'Longitude':144.958897}])
sensor4

,Latitude,Longitude
0,-37.810979,144.958897


### Nearest sensors

In [56]:
# Find Sensor information and Distance

## Change sensor
df4 = pd.concat([pd.DataFrame(within(sensor4, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor4, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [57]:
# Nearest 3 sensors
df4 = df4.sort_values('distance').iloc[0:4,:]
df4 = df4.reset_index(drop=True)
df4.head(3)

,sensor_id,latitude,longitude,distance
0,13.0,-37.812397,144.956526,0.261459
1,56.0,-37.812348,144.961533,0.277421
2,51.0,-37.808418,144.959063,0.284614


In [58]:
df_nearest = df3_new[(df3_new['sensor_id'] == df4['sensor_id'][0]) |
                 (df3_new['sensor_id'] == df4['sensor_id'][1]) |
                 ((df3_new['sensor_id'] == df4['sensor_id'][2]))]

In [59]:
d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['hourly_counts']*(df4['distance'][0]/df4['distance'].sum())
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][1]]['hourly_counts']*(df4['distance'][1]/df4['distance'].sum())
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][2]]['hourly_counts']*(df4['distance'][2]/df4['distance'].sum())

In [60]:
## Change sensor

sensor_d = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

In [61]:
sensor_d.head(5)

,date_time,hourly_counts
0,17/02/2019 19:00,191.0
1,17/02/2019 20:00,179.0
2,17/02/2019 21:00,140.0
3,17/02/2019 22:00,90.0
4,17/02/2019 23:00,54.0


In [62]:
## Change sensor

sensor_d['latitude']= sensor4['Latitude'][0]
sensor_d['longitude']= sensor4['Longitude'][0]
sensor_d['sensor_name']= 'Queen ST - Virtual'
sensor_d['sensor_id']= 103
sensor_d.head()

,date_time,hourly_counts,latitude,longitude,sensor_name,sensor_id
0,17/02/2019 19:00,191.0,-37.810979,144.958897,Queen ST - Virtual,103
1,17/02/2019 20:00,179.0,-37.810979,144.958897,Queen ST - Virtual,103
2,17/02/2019 21:00,140.0,-37.810979,144.958897,Queen ST - Virtual,103
3,17/02/2019 22:00,90.0,-37.810979,144.958897,Queen ST - Virtual,103
4,17/02/2019 23:00,54.0,-37.810979,144.958897,Queen ST - Virtual,103


### Append Existing DF

In [63]:
## Change sensor

df3_new = df3_new.append(sensor_d, sort=False).reset_index(drop=True)

In [64]:
## Change sensor

df2 = df2.append(sensor_d.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

### <font color='blue'>Sensor E - Hyatt</font>

In [65]:
sensor5 = pd.DataFrame([{'Latitude':-37.81550,'Longitude':144.97005}])
sensor5

,Latitude,Longitude
0,-37.8155,144.97005


### Nearest sensors

In [66]:
# Find Sensor information and Distance

## Change sensor
df4 = pd.concat([pd.DataFrame(within(sensor5, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor5, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [67]:
# Nearest 3 sensors
df4 = df4.sort_values('distance').iloc[0:4,:]
df4 = df4.reset_index(drop=True)
df4.head(3)

,sensor_id,latitude,longitude,distance
0,39.0,-37.813797,144.969957,0.189142
1,18.0,-37.813449,144.973053,0.348979
2,17.0,-37.813625,144.973236,0.349267


In [68]:
df_nearest = df3_new[(df3_new['sensor_id'] == df4['sensor_id'][0]) |
                 (df3_new['sensor_id'] == df4['sensor_id'][1]) |
                 ((df3_new['sensor_id'] == df4['sensor_id'][2]))]

In [69]:
d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['hourly_counts']*(df4['distance'][0]/df4['distance'].sum())
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][1]]['hourly_counts']*(df4['distance'][1]/df4['distance'].sum())
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][2]]['hourly_counts']*(df4['distance'][2]/df4['distance'].sum())

In [70]:
## Change sensor

sensor_e = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

In [71]:
sensor_e.head(5)

,date_time,hourly_counts
0,17/02/2019 19:00,94.0
1,17/02/2019 20:00,84.0
2,17/02/2019 21:00,64.0
3,17/02/2019 22:00,47.0
4,17/02/2019 23:00,33.0


In [72]:
## Change sensor

sensor_e['latitude']= sensor5['Latitude'][0]
sensor_e['longitude']= sensor5['Longitude'][0]
sensor_e['sensor_name']= 'Hyatt - Virtual'
sensor_e['sensor_id']= 104
sensor_e.head()

,date_time,hourly_counts,latitude,longitude,sensor_name,sensor_id
0,17/02/2019 19:00,94.0,-37.8155,144.97005,Hyatt - Virtual,104
1,17/02/2019 20:00,84.0,-37.8155,144.97005,Hyatt - Virtual,104
2,17/02/2019 21:00,64.0,-37.8155,144.97005,Hyatt - Virtual,104
3,17/02/2019 22:00,47.0,-37.8155,144.97005,Hyatt - Virtual,104
4,17/02/2019 23:00,33.0,-37.8155,144.97005,Hyatt - Virtual,104


### Append Existing DF

In [73]:
## Change sensor

df3_new = df3_new.append(sensor_e, sort=False).reset_index(drop=True)

In [74]:
## Change sensor

df2 = df2.append(sensor_e.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

### <font color='blue'>Sensor F - Exhibition St</font>

In [75]:
sensor6 = pd.DataFrame([{'Latitude':-37.808234,'Longitude':144.968757}])
sensor6

,Latitude,Longitude
0,-37.808234,144.968757


### Nearest sensors

In [76]:
# Find Sensor information and Distance

## Change sensor
df4 = pd.concat([pd.DataFrame(within(sensor6, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor6, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [77]:
# Nearest 3 sensors
df4 = df4.sort_values('distance').iloc[0:4,:]
df4 = df4.reset_index(drop=True)
df4.head(3)

,sensor_id,latitude,longitude,distance
0,40.0,-37.809993,144.972276,0.366274
1,30.0,-37.811218,144.966568,0.383255
2,20.0,-37.811729,144.968247,0.390529


In [78]:
df_nearest = df3_new[(df3_new['sensor_id'] == df4['sensor_id'][0]) |
                 (df3_new['sensor_id'] == df4['sensor_id'][1]) |
                 ((df3_new['sensor_id'] == df4['sensor_id'][2]))]

In [79]:
d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['hourly_counts']*(df4['distance'][0]/df4['distance'].sum())
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][1]]['hourly_counts']*(df4['distance'][1]/df4['distance'].sum())
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][2]]['hourly_counts']*(df4['distance'][2]/df4['distance'].sum())

In [80]:
## Change sensor

sensor_f = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

In [81]:
sensor_f.head(5)

,date_time,hourly_counts
0,17/02/2019 19:00,451.0
1,17/02/2019 20:00,363.0
2,17/02/2019 21:00,297.0
3,17/02/2019 22:00,268.0
4,17/02/2019 23:00,115.0


In [82]:
## Change sensor

sensor_f['latitude']= sensor6['Latitude'][0]
sensor_f['longitude']= sensor6['Longitude'][0]
sensor_f['sensor_name']= 'Exhibition St - Virtual'
sensor_f['sensor_id']= 105
sensor_f.head()

,date_time,hourly_counts,latitude,longitude,sensor_name,sensor_id
0,17/02/2019 19:00,451.0,-37.808234,144.968757,Exhibition St - Virtual,105
1,17/02/2019 20:00,363.0,-37.808234,144.968757,Exhibition St - Virtual,105
2,17/02/2019 21:00,297.0,-37.808234,144.968757,Exhibition St - Virtual,105
3,17/02/2019 22:00,268.0,-37.808234,144.968757,Exhibition St - Virtual,105
4,17/02/2019 23:00,115.0,-37.808234,144.968757,Exhibition St - Virtual,105


### Append Existing DF

In [83]:
## Change sensor

df3_new = df3_new.append(sensor_f, sort=False).reset_index(drop=True)

In [84]:
## Change sensor

df2 = df2.append(sensor_f.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

### <font color='blue'>Sensor G - La Trobe St</font>

In [85]:
sensor7 = pd.DataFrame([{'Latitude':-37.81319,'Longitude':144.95174}])
sensor7

,Latitude,Longitude
0,-37.81319,144.95174


### Nearest sensors

In [86]:
# Find Sensor information and Distance

## Change sensor
df4 = pd.concat([pd.DataFrame(within(sensor7, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor7, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [87]:
# Nearest 3 sensors
df4 = df4.sort_values('distance').iloc[0:4,:]
df4 = df4.reset_index(drop=True)
df4.head(3)

,sensor_id,latitude,longitude,distance
0,13.0,-37.812397,144.956526,0.430526
1,58.0,-37.816861,144.953581,0.438484
2,57.0,-37.817673,144.950256,0.514507


In [88]:
df_nearest = df3_new[(df3_new['sensor_id'] == df4['sensor_id'][0]) |
                 (df3_new['sensor_id'] == df4['sensor_id'][1]) |
                 ((df3_new['sensor_id'] == df4['sensor_id'][2]))]

In [89]:
d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['hourly_counts']*(df4['distance'][0]/df4['distance'].sum())
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][1]]['hourly_counts']*(df4['distance'][1]/df4['distance'].sum())
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][2]]['hourly_counts']*(df4['distance'][2]/df4['distance'].sum())

In [90]:
## Change sensor

sensor_g = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

In [91]:
sensor_g.head(5)

,date_time,hourly_counts
0,17/02/2019 19:00,811.0
1,17/02/2019 20:00,231.0
2,17/02/2019 21:00,172.0
3,17/02/2019 22:00,119.0
4,17/02/2019 23:00,95.0


In [92]:
## Change sensor

sensor_g['latitude']= sensor7['Latitude'][0]
sensor_g['longitude']= sensor7['Longitude'][0]
sensor_g['sensor_name']= 'La Trobe St - Virtual'
sensor_g['sensor_id']= 106
sensor_g.head()

,date_time,hourly_counts,latitude,longitude,sensor_name,sensor_id
0,17/02/2019 19:00,811.0,-37.81319,144.95174,La Trobe St - Virtual,106
1,17/02/2019 20:00,231.0,-37.81319,144.95174,La Trobe St - Virtual,106
2,17/02/2019 21:00,172.0,-37.81319,144.95174,La Trobe St - Virtual,106
3,17/02/2019 22:00,119.0,-37.81319,144.95174,La Trobe St - Virtual,106
4,17/02/2019 23:00,95.0,-37.81319,144.95174,La Trobe St - Virtual,106


### Append Existing DF

In [93]:
## Change sensor

df3_new = df3_new.append(sensor_g, sort=False).reset_index(drop=True)

In [94]:
## Change sensor

df2 = df2.append(sensor_g.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

### <font color='blue'>Sensor H - William St</font>

In [95]:
sensor8 = pd.DataFrame([{'Latitude':-37.81563,'Longitude':144.95823}])
sensor8

,Latitude,Longitude
0,-37.81563,144.95823


### Nearest sensors

In [96]:
# Find Sensor information and Distance

## Change sensor
df4 = pd.concat([pd.DataFrame(within(sensor8, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor8, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [97]:
# Nearest 3 sensors
df4 = df4.sort_values('distance').iloc[0:4,:]
df4 = df4.reset_index(drop=True)
df4.head(3)

,sensor_id,latitude,longitude,distance
0,101.0,-37.817771,144.957951,0.238902
1,36.0,-37.816525,144.961211,0.280628
2,102.0,-37.814470,144.961970,0.353591


In [98]:
df_nearest = df3_new[(df3_new['sensor_id'] == df4['sensor_id'][0]) |
                 (df3_new['sensor_id'] == df4['sensor_id'][1]) |
                 ((df3_new['sensor_id'] == df4['sensor_id'][2]))]

In [99]:
d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['hourly_counts']*(df4['distance'][0]/df4['distance'].sum())
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][1]]['hourly_counts']*(df4['distance'][1]/df4['distance'].sum())
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][2]]['hourly_counts']*(df4['distance'][2]/df4['distance'].sum())

In [100]:
## Change sensor

sensor_h = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

In [101]:
sensor_h.head(5)

,date_time,hourly_counts
0,17/02/2019 19:00,222.0
1,17/02/2019 20:00,175.0
2,17/02/2019 21:00,156.0
3,17/02/2019 22:00,99.0
4,17/02/2019 23:00,63.0


In [102]:
## Change sensor

sensor_h['latitude']= sensor8['Latitude'][0]
sensor_h['longitude']= sensor8['Longitude'][0]
sensor_h['sensor_name']= 'Willam St - Virtual'
sensor_h['sensor_id']= 107
sensor_h.head()

,date_time,hourly_counts,latitude,longitude,sensor_name,sensor_id
0,17/02/2019 19:00,222.0,-37.81563,144.95823,Willam St - Virtual,107
1,17/02/2019 20:00,175.0,-37.81563,144.95823,Willam St - Virtual,107
2,17/02/2019 21:00,156.0,-37.81563,144.95823,Willam St - Virtual,107
3,17/02/2019 22:00,99.0,-37.81563,144.95823,Willam St - Virtual,107
4,17/02/2019 23:00,63.0,-37.81563,144.95823,Willam St - Virtual,107


### Append Existing DF

In [103]:
## Change sensor

df3_new = df3_new.append(sensor_h, sort=False).reset_index(drop=True)

In [104]:
## Change sensor

df2 = df2.append(sensor_h.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

### <font color='blue'>Sensor I - Sea Life </font>

In [105]:
sensor9 = pd.DataFrame([{'Latitude':-37.82038,'Longitude':144.95746}])
sensor9

,Latitude,Longitude
0,-37.82038,144.95746


### Nearest sensors

In [106]:
# Find Sensor information and Distance

## Change sensor
df4 = pd.concat([pd.DataFrame(within(sensor9, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor9, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [107]:
# Nearest 3 sensors
df4 = df4.sort_values('distance').iloc[0:4,:]
df4 = df4.reset_index(drop=True)
df4.head(3)

,sensor_id,latitude,longitude,distance
0,101.0,-37.817771,144.957951,0.292790
1,23.0,-37.819093,144.954527,0.295103
2,24.0,-37.818880,144.954492,0.309876


In [108]:
df_nearest = df3_new[(df3_new['sensor_id'] == df4['sensor_id'][0]) |
                 (df3_new['sensor_id'] == df4['sensor_id'][1]) |
                 ((df3_new['sensor_id'] == df4['sensor_id'][2]))]

In [109]:
d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['hourly_counts']*(df4['distance'][0]/df4['distance'].sum())
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][1]]['hourly_counts']*(df4['distance'][1]/df4['distance'].sum())
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][2]]['hourly_counts']*(df4['distance'][2]/df4['distance'].sum())

In [110]:
## Change sensor

sensor_i = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

sensor_i.head(5)

,date_time,hourly_counts
0,17/02/2019 19:00,378.0
1,17/02/2019 20:00,291.0
2,17/02/2019 21:00,254.0
3,17/02/2019 22:00,161.0
4,17/02/2019 23:00,111.0


In [111]:
## Change sensor

sensor_i['latitude']= sensor9['Latitude'][0]
sensor_i['longitude']= sensor9['Longitude'][0]
sensor_i['sensor_name']= 'Sea Life - Virtual'
sensor_i['sensor_id']= 108
sensor_i.head()

,date_time,hourly_counts,latitude,longitude,sensor_name,sensor_id
0,17/02/2019 19:00,378.0,-37.82038,144.95746,Sea Life - Virtual,108
1,17/02/2019 20:00,291.0,-37.82038,144.95746,Sea Life - Virtual,108
2,17/02/2019 21:00,254.0,-37.82038,144.95746,Sea Life - Virtual,108
3,17/02/2019 22:00,161.0,-37.82038,144.95746,Sea Life - Virtual,108
4,17/02/2019 23:00,111.0,-37.82038,144.95746,Sea Life - Virtual,108


### Append Existing DF

In [112]:
## Change sensor

df3_new = df3_new.append(sensor_i, sort=False).reset_index(drop=True)

In [113]:
## Change sensor

df2 = df2.append(sensor_i.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

### <font color='blue'>Sensor J - Immigration Museum </font>

In [114]:
sensor10 = pd.DataFrame([{'Latitude':-37.81934,'Longitude':144.96075}])
sensor10

,Latitude,Longitude
0,-37.81934,144.96075


### Nearest sensors

In [129]:
# Find Sensor information and Distance

## Change sensor
df4 = pd.concat([pd.DataFrame(within(sensor10, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor10, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [132]:
# Nearest 3 sensors
df4 = df4.sort_values('distance').iloc[1:4,:]
df4 = df4.reset_index(drop=True)
df4.head(4)

,sensor_id,latitude,longitude,distance
0,101.0,-37.817771,144.957951,0.301768
1,108.0,-37.820380,144.957460,0.311827
2,36.0,-37.816525,144.961211,0.315037


In [133]:
df_nearest = df3_new[(df3_new['sensor_id'] == df4['sensor_id'][0]) |
                 (df3_new['sensor_id'] == df4['sensor_id'][1]) |
                 ((df3_new['sensor_id'] == df4['sensor_id'][2]))]

In [134]:
d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['hourly_counts']*(df4['distance'][0]/df4['distance'].sum())
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][1]]['hourly_counts']*(df4['distance'][1]/df4['distance'].sum())
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][2]]['hourly_counts']*(df4['distance'][2]/df4['distance'].sum())

In [135]:
## Change sensor

sensor_j = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

sensor_j.head(5)

,date_time,hourly_counts
0,17/02/2019 19:00,335.0
1,17/02/2019 20:00,256.0
2,17/02/2019 21:00,237.0
3,17/02/2019 22:00,148.0
4,17/02/2019 23:00,93.0


In [136]:
## Change sensor

sensor_j['latitude']= sensor10['Latitude'][0]
sensor_j['longitude']= sensor10['Longitude'][0]
sensor_j['sensor_name']= 'Immigration Museum - Virtual'
sensor_j['sensor_id']= 109
sensor_j.head()

,date_time,hourly_counts,latitude,longitude,sensor_name,sensor_id
0,17/02/2019 19:00,335.0,-37.81934,144.96075,Immigration Museum - Virtual,109
1,17/02/2019 20:00,256.0,-37.81934,144.96075,Immigration Museum - Virtual,109
2,17/02/2019 21:00,237.0,-37.81934,144.96075,Immigration Museum - Virtual,109
3,17/02/2019 22:00,148.0,-37.81934,144.96075,Immigration Museum - Virtual,109
4,17/02/2019 23:00,93.0,-37.81934,144.96075,Immigration Museum - Virtual,109


### Append Existing DF

In [137]:
## Change sensor

df3_new = df3_new.append(sensor_j, sort=False).reset_index(drop=True)

In [138]:
## Change sensor

df2 = df2.append(sensor_j.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

### <font color='blue'>Sensor K - RMIT </font>

In [139]:
sensor11 = pd.DataFrame([{'Latitude':-37.80892,'Longitude':144.96634}])
sensor11

,Latitude,Longitude
0,-37.80892,144.96634


### Nearest sensors

In [153]:
# Find Sensor information and Distance

## Change sensor
df4 = pd.concat([pd.DataFrame(within(sensor11, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor11, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [154]:
# Nearest 3 sensors
df4 = df4.sort_values('distance').iloc[[0,3,4],:]
df4 = df4.reset_index(drop=True)
df4.head(3)

,sensor_id,latitude,longitude,distance
0,105.0,-37.808234,144.968757,0.226051
1,59.0,-37.808256,144.963049,0.299052
2,20.0,-37.811729,144.968247,0.354123


In [155]:
df_nearest = df3_new[(df3_new['sensor_id'] == df4['sensor_id'][0]) |
                 (df3_new['sensor_id'] == df4['sensor_id'][1]) |
                 ((df3_new['sensor_id'] == df4['sensor_id'][2]))]

In [156]:
d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['hourly_counts']*(df4['distance'][0]/df4['distance'].sum())
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][1]]['hourly_counts']*(df4['distance'][1]/df4['distance'].sum())
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][2]]['hourly_counts']*(df4['distance'][2]/df4['distance'].sum())

In [158]:
## Change sensor
sensor_k = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

sensor_k.head(5)

,date_time,hourly_counts
0,17/02/2019 19:00,707.0
1,17/02/2019 20:00,546.0
2,17/02/2019 21:00,457.0
3,17/02/2019 22:00,305.0
4,17/02/2019 23:00,158.0


In [159]:
## Change sensor

sensor_k['latitude']= sensor11['Latitude'][0]
sensor_k['longitude']= sensor11['Longitude'][0]
sensor_k['sensor_name']= 'RMIT - Virtual'
sensor_k['sensor_id']= 110
sensor_k.head()

,date_time,hourly_counts,latitude,longitude,sensor_name,sensor_id
0,17/02/2019 19:00,707.0,-37.80892,144.96634,RMIT - Virtual,110
1,17/02/2019 20:00,546.0,-37.80892,144.96634,RMIT - Virtual,110
2,17/02/2019 21:00,457.0,-37.80892,144.96634,RMIT - Virtual,110
3,17/02/2019 22:00,305.0,-37.80892,144.96634,RMIT - Virtual,110
4,17/02/2019 23:00,158.0,-37.80892,144.96634,RMIT - Virtual,110


### Append Existing DF

In [160]:
## Change sensor

df3_new = df3_new.append(sensor_k, sort=False).reset_index(drop=True)

In [161]:
## Change sensor

df2 = df2.append(sensor_k.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

## Save

In [162]:
df2.to_csv('df2.csv')

In [164]:
df3_new.to_csv('pedestrian_temp.csv', index=False)

# <font color='red'> For Each Gallery </font>

In [20]:
## Change sensor
df4 = pd.concat([pd.DataFrame(within(sensor11, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor11, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [5]:
from datetime import date
import calendar# Find Sensor information and Distance

In [6]:
df = pd.read_csv("pedestrian_temp.csv")
gallery = pd.read_csv("../data/Gallery.csv")
df2 = pd.read_csv('df2.csv')

In [7]:
# Find weekday
df['date_time'] = pd.to_datetime(df['date_time'])

In [8]:
my_day_list = []
for i in range(len(df)):
    my_date = df['date_time'][i]
    my_day = calendar.day_name[my_date.weekday()]
    my_day_list.append(my_day)

In [9]:
# Append Column
df['Day'] = my_day_list

In [10]:
df.head()

,date_time,hourly_counts,sensor_id,sensor_name,latitude,longitude,Day
0,2019-02-17 19:00:00,1513.0,4,Town Hall (West),-37.81488,144.966088,Sunday
1,2019-02-17 20:00:00,1234.0,4,Town Hall (West),-37.81488,144.966088,Sunday
2,2019-02-17 21:00:00,1026.0,4,Town Hall (West),-37.81488,144.966088,Sunday
3,2019-02-17 22:00:00,693.0,4,Town Hall (West),-37.81488,144.966088,Sunday
4,2019-02-17 23:00:00,388.0,4,Town Hall (West),-37.81488,144.966088,Sunday


In [11]:
def get_ped(gallery, pedestrian, i):

    gallery_sensor = pd.DataFrame([{'Latitude':gallery['Latitude'][i],'Longitude': gallery['Longitude'][i]}])

    # Find Nearest 3 sensors
    df_temp = pd.concat([pd.DataFrame(within(gallery_sensor, df2, 1.5, 0)).reset_index(drop=True),
               pd.DataFrame(within_d(gallery_sensor, df2, 1.5, 0)).reset_index(drop=True)], axis=1)
    df_temp.rename(columns={0:'distance'}, inplace=True)

    # Get Nearest 3 sensors
    df_temp = df_temp.sort_values('distance').iloc[0:3,:]
    df_temp = df_temp.reset_index(drop=True)

    # DF Nearest
    df_nearest = pedestrian[(pedestrian['sensor_id'] == df_temp['sensor_id'][0]) |
                 (pedestrian['sensor_id'] == df_temp['sensor_id'][1]) |
                 ((pedestrian['sensor_id'] == df_temp['sensor_id'][2]))]

    d1 = df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][0]]['hourly_counts']*(df_temp['distance'][0]/df_temp['distance'].sum())
    d2 = df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][1]]['hourly_counts']*(df_temp['distance'][1]/df_temp['distance'].sum())
    d3 = df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][2]]['hourly_counts']*(df_temp['distance'][2]/df_temp['distance'].sum())

    ## Change sensor
    sensor_interest = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

    ## Change sensor
    sensor_interest['latitude']= gallery_sensor['Latitude'][0]
    sensor_interest['longitude']= gallery_sensor['Longitude'][0]
    sensor_interest['sensor_name']= gallery['Gallery Name'][i]

    my_day_list = []

    sensor_interest['date_time'] = pd.to_datetime(sensor_interest['date_time'])

    for i in range(len(sensor_interest)):
        my_date = sensor_interest['date_time'][i]
        my_day = calendar.day_name[my_date.weekday()]
        my_day_list.append(my_day)

    sensor_interest['Day'] = my_day_list
    

    return sensor_interest

In [12]:
def for_eachday(df):

    each = []
    day_list = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday', 'Saturday']
    
    # Change time to int
    time = df['date_time'].apply(lambda x: int(x.strftime('%H')))
    df['time'] = time
        
    # Loop over days
    for i in range (7):
        # 10AM - 5PM
        dday = round(df[(df['Day']==day_list[i]) & (df['time'] < 17) & (df['time'] > 10) ]['hourly_counts'].mean())
        each.append(dday)
        
    return each

In [306]:
# Find average for each gallery
average_list = []

for n in range(0,len(gallery)):
    average = for_eachday(get_ped(gallery, df, i=n))
    average_list.append(average)

In [316]:
# Concat
past = pd.concat([gallery, pd.DataFrame(average_list, columns=day_list)], axis=1)
past.to_csv('past.csv')
past

,Gallery Name,Latitude,Longitude,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
0,Old Melbourne Gaol Crime,-37.807764,144.965464,569.0,945.0,850.0,1017.0,758.0,903.0,796.0
1,Victorian Arts Centre,-37.821995,144.968837,2345.0,1572.0,1835.0,1672.0,1897.0,1884.0,2012.0
2,The Ian Potter Centre: NGV Australia,-37.817483,144.969899,658.0,785.0,810.0,806.0,867.0,923.0,583.0
3,Thoroughbred Racing Gallery,-37.818866,144.983970,202.0,598.0,538.0,636.0,576.0,740.0,317.0
4,Fox Classic Car Collection,-37.821374,144.948497,158.0,754.0,671.0,785.0,692.0,876.0,335.0
5,Koorie Heritage Trust Inc,-37.813385,144.954028,602.0,1034.0,1003.0,1077.0,1017.0,1152.0,777.0
6,Australian Centre For The Moving Image (ACMI),-37.817611,144.969070,1289.0,1386.0,1439.0,1445.0,1506.0,1703.0,1323.0
7,Blindside Gallery,-37.816711,144.966812,2130.0,2602.0,2641.0,2704.0,2624.0,2844.0,2519.0
8,Flinders Lane Gallery,-37.815715,144.970033,1613.0,2011.0,2066.0,2076.0,2043.0,2258.0,1964.0
9,Mailbox Art Space,-37.815758,144.969861,1557.0,1961.0,2011.0,2024.0,1990.0,2202.0,1900.0


## Any Location 

In [1]:
click = [144.96629694772997, -37.813251541664975]

In [10]:
from datetime import date
import calendar# Find Sensor information and Distance
import pandas as pd
import geopy
from geopy import distance


df = pd.read_csv("../static/data/pedestrian_temp.csv")
df2 = pd.read_csv('../static/data/sensors.csv')


def within(df, target, radius, g):
    
    temp = []
    
    for x in range(len(target)):
        # Use try-except for error handling
        try:
            # Find coordinate1,2 calculate distance in km
            coords_1 = (df["Latitude"][g], df["Longitude"][g])
            coords_2 = (target["latitude"][x], target["longitude"][x])
            dist = geopy.distance.distance(coords_1, coords_2).km

            if dist <= radius:
                temp.append(target.iloc[x])
        except:
            pass
    return temp


def within_d(df, target, radius, g):
    
    distances = []
    
    for x in range(len(target)):
        # Use try-except for error handling
        try:
            # Find coordinate1,2 calculate distance in km
            coords_1 = (df["Latitude"][g], df["Longitude"][g])
            coords_2 = (target["latitude"][x], target["longitude"][x])
            dist = geopy.distance.distance(coords_1, coords_2).km

            if dist <= radius:
                distances.append(dist)
        except:
            pass
    return distances




def get_ped_any(click, pedestrian):

    click_sensor = pd.DataFrame([{'Latitude':click[1],'Longitude': click[0]}])

    # Find Nearest 3 sensors
    df_temp = pd.concat([pd.DataFrame(within(click_sensor, df2, 1.5, 0)).reset_index(drop=True),
               pd.DataFrame(within_d(click_sensor, df2, 1.5, 0)).reset_index(drop=True)], axis=1)
    df_temp.rename(columns={0:'distance'}, inplace=True)

    # Get Nearest 3 sensors
    df_temp = df_temp.sort_values('distance').iloc[0:3,:]
    df_temp = df_temp.reset_index(drop=True)

    # DF Nearest
    df_nearest = pedestrian[(pedestrian['sensor_id'] == df_temp['sensor_id'][0]) |
                 (pedestrian['sensor_id'] == df_temp['sensor_id'][1]) |
                 ((pedestrian['sensor_id'] == df_temp['sensor_id'][2]))]

    d1 = df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][0]]['hourly_counts']*(df_temp['distance'][0]/df_temp['distance'].sum())
    d2 = df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][1]]['hourly_counts']*(df_temp['distance'][1]/df_temp['distance'].sum())
    d3 = df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][2]]['hourly_counts']*(df_temp['distance'][2]/df_temp['distance'].sum())

    ## Change sensor
    sensor_interest = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

    ## Change sensor
    sensor_interest['latitude']= click_sensor['Latitude'][0]
    sensor_interest['longitude']= click_sensor['Longitude'][0]

    my_day_list = []

    sensor_interest['date_time'] = pd.to_datetime(sensor_interest['date_time'])

    for i in range(len(sensor_interest)):
        my_date = sensor_interest['date_time'][i]
        my_day = calendar.day_name[my_date.weekday()]
        my_day_list.append(my_day)

    sensor_interest['Day'] = my_day_list
    

    return sensor_interest



,date_time,hourly_counts,latitude,longitude,Day
0,2019-02-17 19:00:00,1035.0,-37.813252,144.966297,Sunday
1,2019-02-17 20:00:00,765.0,-37.813252,144.966297,Sunday
2,2019-02-17 21:00:00,547.0,-37.813252,144.966297,Sunday
3,2019-02-17 22:00:00,356.0,-37.813252,144.966297,Sunday
4,2019-02-17 23:00:00,146.0,-37.813252,144.966297,Sunday
5,2019-02-18 00:00:00,80.0,-37.813252,144.966297,Monday
6,2019-02-18 01:00:00,61.0,-37.813252,144.966297,Monday
7,2019-02-18 02:00:00,31.0,-37.813252,144.966297,Monday
8,2019-02-18 03:00:00,26.0,-37.813252,144.966297,Monday
9,2019-02-18 04:00:00,13.0,-37.813252,144.966297,Monday


In [13]:
print(get_ped_any(click, df).head())

            date_time  hourly_counts   latitude   longitude     Day
0 2019-02-17 19:00:00         1035.0 -37.813252  144.966297  Sunday
1 2019-02-17 20:00:00          765.0 -37.813252  144.966297  Sunday
2 2019-02-17 21:00:00          547.0 -37.813252  144.966297  Sunday
3 2019-02-17 22:00:00          356.0 -37.813252  144.966297  Sunday
4 2019-02-17 23:00:00          146.0 -37.813252  144.966297  Sunday
